# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 30 2022 11:34AM,240842,15,30009352,"Alliance Pharma, Inc.",Released
1,Jun 30 2022 11:34AM,240842,15,30009353,"Alliance Pharma, Inc.",Released
2,Jun 30 2022 11:34AM,240842,15,30009354,"Alliance Pharma, Inc.",Released
3,Jun 30 2022 11:34AM,240842,15,30009355,"Alliance Pharma, Inc.",Released
4,Jun 30 2022 11:34AM,240842,15,30009357,"Alliance Pharma, Inc.",Released
5,Jun 30 2022 11:34AM,240842,15,30009356,"Alliance Pharma, Inc.",Released
6,Jun 30 2022 11:34AM,240842,15,30009358,"Alliance Pharma, Inc.",Released
7,Jun 30 2022 11:34AM,240842,15,30009359,"Alliance Pharma, Inc.",Released
8,Jun 30 2022 11:34AM,240842,15,30009361,"Alliance Pharma, Inc.",Released
9,Jun 30 2022 11:34AM,240842,15,30009360,"Alliance Pharma, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
33,240838,Released,21
34,240840,Executing,1
35,240840,Released,2
36,240841,Released,1
37,240842,Released,19


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
240834,NaN,NaN,1.0
240838,NaN,NaN,21.0
240840,NaN,1.0,2.0
240841,NaN,NaN,1.0
240842,NaN,NaN,19.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240750,17.0,8.0,1.0
240766,0.0,0.0,1.0
240768,0.0,0.0,1.0
240774,0.0,1.0,0.0
240776,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240750,17,8,1
240766,0,0,1
240768,0,0,1
240774,0,1,0
240776,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240750,17,8,1
1,240766,0,0,1
2,240768,0,0,1
3,240774,0,1,0
4,240776,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240750,17,8,1
1,240766,,,1
2,240768,,,1
3,240774,,1,
4,240776,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 30 2022 11:34AM,240842,15,"Alliance Pharma, Inc."
19,Jun 30 2022 11:20AM,240841,20,"Exact-Rx, Inc."
20,Jun 30 2022 10:54AM,240840,19,Dexcel Pharma Technologies Ltd. DBA WDSrx
23,Jun 30 2022 10:47AM,240838,10,ISDIN Corporation
44,Jun 30 2022 10:42AM,240833,10,ISDIN Corporation
50,Jun 30 2022 10:42AM,240834,10,"Methapharm, Inc."
51,Jun 30 2022 10:40AM,240832,10,ISDIN Corporation
88,Jun 30 2022 10:35AM,240831,19,"NAPP Technologies, LLC"
89,Jun 30 2022 10:10AM,240830,20,Alumier Labs Inc.
90,Jun 30 2022 10:03AM,240829,20,Alumier Labs Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 30 2022 11:34AM,240842,15,"Alliance Pharma, Inc.",,,19
1,Jun 30 2022 11:20AM,240841,20,"Exact-Rx, Inc.",,,1
2,Jun 30 2022 10:54AM,240840,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,,1,2
3,Jun 30 2022 10:47AM,240838,10,ISDIN Corporation,,,21
4,Jun 30 2022 10:42AM,240833,10,ISDIN Corporation,,,6
5,Jun 30 2022 10:42AM,240834,10,"Methapharm, Inc.",,,1
6,Jun 30 2022 10:40AM,240832,10,ISDIN Corporation,,,37
7,Jun 30 2022 10:35AM,240831,19,"NAPP Technologies, LLC",,,1
8,Jun 30 2022 10:10AM,240830,20,Alumier Labs Inc.,,,1
9,Jun 30 2022 10:03AM,240829,20,Alumier Labs Inc.,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 30 2022 11:34AM,240842,15,"Alliance Pharma, Inc.",19,,
1,Jun 30 2022 11:20AM,240841,20,"Exact-Rx, Inc.",1,,
2,Jun 30 2022 10:54AM,240840,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,2,1,
3,Jun 30 2022 10:47AM,240838,10,ISDIN Corporation,21,,
4,Jun 30 2022 10:42AM,240833,10,ISDIN Corporation,6,,
5,Jun 30 2022 10:42AM,240834,10,"Methapharm, Inc.",1,,
6,Jun 30 2022 10:40AM,240832,10,ISDIN Corporation,37,,
7,Jun 30 2022 10:35AM,240831,19,"NAPP Technologies, LLC",1,,
8,Jun 30 2022 10:10AM,240830,20,Alumier Labs Inc.,1,,
9,Jun 30 2022 10:03AM,240829,20,Alumier Labs Inc.,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 30 2022 11:34AM,240842,15,"Alliance Pharma, Inc.",19,,
1,Jun 30 2022 11:20AM,240841,20,"Exact-Rx, Inc.",1,,
2,Jun 30 2022 10:54AM,240840,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,2,1,
3,Jun 30 2022 10:47AM,240838,10,ISDIN Corporation,21,,
4,Jun 30 2022 10:42AM,240833,10,ISDIN Corporation,6,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 30 2022 11:34AM,240842,15,"Alliance Pharma, Inc.",19.0,NaN,NaN
1,Jun 30 2022 11:20AM,240841,20,"Exact-Rx, Inc.",1.0,NaN,NaN
2,Jun 30 2022 10:54AM,240840,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,2.0,1.0,NaN
3,Jun 30 2022 10:47AM,240838,10,ISDIN Corporation,21.0,NaN,NaN
4,Jun 30 2022 10:42AM,240833,10,ISDIN Corporation,6.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 30 2022 11:34AM,240842,15,"Alliance Pharma, Inc.",19.0,0.0,0.0
1,Jun 30 2022 11:20AM,240841,20,"Exact-Rx, Inc.",1.0,0.0,0.0
2,Jun 30 2022 10:54AM,240840,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,2.0,1.0,0.0
3,Jun 30 2022 10:47AM,240838,10,ISDIN Corporation,21.0,0.0,0.0
4,Jun 30 2022 10:42AM,240833,10,ISDIN Corporation,6.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1685729,119.0,14.0,0.0
12,481619,0.0,3.0,0.0
15,240842,19.0,0.0,0.0
16,722424,4.0,5.0,0.0
17,240768,1.0,0.0,0.0
18,240813,1.0,0.0,0.0
19,1204103,31.0,22.0,9.0
20,1685602,24.0,16.0,34.0
21,1203910,3.0,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1685729,119.0,14.0,0.0
1,12,481619,0.0,3.0,0.0
2,15,240842,19.0,0.0,0.0
3,16,722424,4.0,5.0,0.0
4,17,240768,1.0,0.0,0.0
5,18,240813,1.0,0.0,0.0
6,19,1204103,31.0,22.0,9.0
7,20,1685602,24.0,16.0,34.0
8,21,1203910,3.0,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,119.0,14.0,0.0
1,12,0.0,3.0,0.0
2,15,19.0,0.0,0.0
3,16,4.0,5.0,0.0
4,17,1.0,0.0,0.0
5,18,1.0,0.0,0.0
6,19,31.0,22.0,9.0
7,20,24.0,16.0,34.0
8,21,3.0,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,119.0
1,12,Released,0.0
2,15,Released,19.0
3,16,Released,4.0
4,17,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,17,18,19,20,21
Status,,,,,,,,,
Completed,0.0,0.0,0.0,0.0,0.0,0.0,9.0,34.0,0.0
Executing,14.0,3.0,0.0,5.0,0.0,0.0,22.0,16.0,2.0
Released,119.0,0.0,19.0,4.0,1.0,1.0,31.0,24.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,17,18,19,20,21
0,Completed,0.0,0.0,0.0,0.0,0.0,0.0,9.0,34.0,0.0
1,Executing,14.0,3.0,0.0,5.0,0.0,0.0,22.0,16.0,2.0
2,Released,119.0,0.0,19.0,4.0,1.0,1.0,31.0,24.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,17,18,19,20,21
0,Completed,0.0,0.0,0.0,0.0,0.0,0.0,9.0,34.0,0.0
1,Executing,14.0,3.0,0.0,5.0,0.0,0.0,22.0,16.0,2.0
2,Released,119.0,0.0,19.0,4.0,1.0,1.0,31.0,24.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()